In [1]:
%cd D:/datasets/fake_news

D:\datasets\fake_news


In [2]:
%pwd

'D:\\datasets\\fake_news'

In [3]:
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import requests

import os
import sys
import re

### Download montag.wtf

In [194]:
articles = []
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
def get_links(i):
    with requests.get(f'https://www.montag.wtf/page/{i}/', headers=headers, timeout=10) as f:
        soup = BeautifulSoup(f.text, features="html.parser")
        for article in soup.findAll(class_="homepage-post-info"):
            for x in article.findAll("a"):
                articles.append(f'https://www.montag.wtf{x["href"]}')

In [195]:
%%time
with ThreadPoolExecutor(27) as executor:
    results = executor.map(get_links, range(1,27))

Wall time: 861 ms


In [196]:
len(articles)

156

In [197]:
def parse(url):
    with requests.get(url, headers=headers) as f:
        soup = BeautifulSoup(f.text, features="html.parser")
    article_text = soup.findAll(class_="content-text")
    for element in article_text:
        words = element.get_text().encode('utf-8')
        with open("montag.txt", "ab") as f:
            f.write(words)

In [198]:
%%time

with ThreadPoolExecutor(100) as executor:
    results = executor.map(parse, articles)

Wall time: 2.02 s


### Download OneZero

In [199]:
from selenium import webdriver
import shutil
import time

In [200]:
url = 'https://onezero.medium.com/latest'
driver = webdriver.Chrome()
driver.get(url)

for _ in range(50):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    time.sleep(.5)

articles = []
for date in driver.find_elements_by_tag_name('time'):
    link = date.find_element_by_xpath('..').get_attribute('href')
    articles.append(link)

driver.quit()

In [29]:
len(articles)

424

In [201]:
for i in range(0, 10):
    articles[i] = re.findall('(.*)\?', articles[i])[0]
for i in range(len(articles)):
    articles[i] += '?format=json'

In [203]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
def parse(url):
    with requests.get(url, headers=headers) as response:
        soup = BeautifulSoup(response.text, features='html.parser')
#     article_text = soup.find(class_='section-inner')
#     with open("onezero.txt", "a") as f:
#         f.write('\n' + article_text.find('h1').get_text() + '\n')
#         f.write(article_text.find('h2').get_text() + '\n')
#         for element in article_text.findAll('p'):
#             words = element.get_text() + '\n'
#             f.write(words)
    image = soup.find(class_='progressiveMedia-image')
    image_url = image['data-src']
    response = requests.get(image_url, stream=True)
    image_name = re.findall('\*(.*)', image_url)[0]
    with open(f'D:/datasets/fake_news/hero_images/one_zero/{image_name}', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [208]:
def parse_json(url):
    with requests.get(url) as response:
        article = json.loads(response.text[16:])
    text = extract_values(article, 'text')
    with open("onezero.txt", "ab") as f:
        for paragraph in text:
            out = paragraph + '\n'
            f.write(out.encode('utf-8'))

In [209]:
%%time

with ThreadPoolExecutor(100) as executor:
    results = executor.map(parse_json, articles)

Wall time: 6.02 s


### Download Futura

In [110]:
from selenium import webdriver
import shutil
import time
import json

In [210]:
url = 'https://medium.com/short-sci-fi-stories/latest'
driver = webdriver.Chrome()
driver.get(url)

for _ in range(50):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    time.sleep(.5)

articles = []
for date in driver.find_elements_by_tag_name('time'):
    link = date.find_element_by_xpath('..').get_attribute('href')
    articles.append(link)

driver.quit()

In [211]:
len(articles)

89

In [212]:
for i in range(0, 10):
    articles[i] = re.findall('(.*)\?', articles[i])[0]
for i in range(len(articles)):
    articles[i] += '?format=json'

In [213]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [214]:
def parse_json(url):
    with requests.get(url) as response:
        article = json.loads(response.text[16:])
    text = extract_values(article, 'text')
    with open("futura.txt", "ab") as f:
        for paragraph in text:
            out = paragraph + '\n'
            f.write(out.encode('utf-8'))

In [215]:
%%time

with ThreadPoolExecutor(80) as executor:
    results = executor.map(parse_json, articles)

Wall time: 1.98 s
